In [1]:
import pandas as pd

# = = =  = = =  = = =

from urllib.parse import urlparse, parse_qs, urlencode

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

In [2]:
print('Data Loading...')
list_series_error, df_input = [], pd.read_excel('./file/1.page.xlsx',
                                                header=0,
                                                dtype=str).fillna('')
print('Loading Done !')
print()

df_input['No'] = df_input['No'].astype(int)
df_input['SKU Count'] = df_input['SKU Count'].astype(int)
df_input['Page'] = df_input['Page'].astype(int)

crawler_remain = len(df_input)

print(f'总数量：{crawler_remain}')
print()

df_input

Data Loading...
Loading Done !

总数量：2



,No,Url,SKU Count,Page
0,1,https://www.standardbrand.com/en/ecatalog?part...,450,5
1,2,https://www.standardbrand.com/en/ecatalog?part...,212,3


In [3]:
crawler_index, crawler_series = next(df_input.iterrows())

crawler_index, crawler_series

(0,
 No                                                           1
 Url          https://www.standardbrand.com/en/ecatalog?part...
 SKU Count                                                  450
 Page                                                         5
 Name: 0, dtype: object)

In [4]:
import time

url_parse = urlparse(crawler_series['Url'])

dict_param = parse_qs(url_parse.query)

crawler_cycle, list_dict = 0, []
while len(list_dict) < crawler_series['SKU Count']:
    crawler_cycle += 1

    for page in range(crawler_series['Page']):

        crawler_retry = 0
        while True:
            crawler_retry += 1
            print(crawler_cycle, page + 1, crawler_retry)
            try:
                url_request, url_check = f'''https://ecatalog.smpcorp.com/V2/STD/api/part/partsearch?filter={dict_param['part'][0]}&filterType={dict_param['search'][0] if 'search' in dict_param else 'n'}&searchType={dict_param['type'][0] if 'type' in dict_param else 'null'}&imageSize=80&start={page * 96}&limit=96&sort=3&catFilter=-All-&yearFilter=-All-&makeFilter=-All-&modelFilter=-All-&engineFilter=-All-&attrCodeFilter=-All-&attrValueFilter=-All-&plkEngineMakeFilter=-All-&plkEngineModelFilter=-All-&plkEngineDispFilter=-All-''', crawler_series['Url']

                resp = requests.get(url_request,
                                    headers=get_header(),
                                    proxies=get_proxy(),
                                    timeout=(5, 15))

                if resp.status_code == 200:
                    list_dict_temp = resp.json()

                    if list_dict_temp:
                        break
            except KeyboardInterrupt:
                break
            except:
                time.sleep(3)
                continue

        # = = =  = = =  = = =

        list_dict.extend(list_dict_temp)

        # = = =  = = =  = = =

        print(f' - {len(list_dict)}/{crawler_series['SKU Count']}')

        # = = =  = = =  = = =

        if len(list_dict) == crawler_series['SKU Count']:
            break

len(list_dict)

1 1 1
 - 96/450
1 2 1
 - 192/450
1 3 1
 - 288/450
1 4 1
 - 384/450
1 5 1
 - 450/450


450

In [5]:
list_dict

[{'rowId': 1,
  'site': 'STD',
  'webBase': 'APK100',
  'brand': 'STD',
  'basePart': 'APK100',
  'partDesc_en': 'Accelerator Pedal Sensor',
  'partLongDesc_en': 'Accelerator Pedal Sensor',
  'pop': 'D',
  'partComment': None,
  'qtyEach': 1,
  'webColumn_Id': None,
  'categoryName_en': None,
  'vehicleId': None,
  'mfg': None,
  'mfG_Part': None,
  'maxRows': 450,
  'brandLink': None,
  'imageUrl': None},
 {'rowId': 2,
  'site': 'STD',
  'webBase': 'APK101',
  'brand': 'STD',
  'basePart': 'APK101',
  'partDesc_en': 'Accelerator Pedal Sensor',
  'partLongDesc_en': 'Accelerator Pedal Sensor',
  'pop': 'C',
  'partComment': None,
  'qtyEach': 1,
  'webColumn_Id': None,
  'categoryName_en': None,
  'vehicleId': None,
  'mfg': None,
  'mfG_Part': None,
  'maxRows': 450,
  'brandLink': None,
  'imageUrl': None},
 {'rowId': 3,
  'site': 'STD',
  'webBase': 'APK102',
  'brand': 'STD',
  'basePart': 'APK102',
  'partDesc_en': 'Accelerator Pedal Sensor',
  'partLongDesc_en': 'Accelerator Pedal

In [6]:
print('Data Outputting...')
df_correct = pd.DataFrame([{'No': dict_['rowId'],
                            'Part Number': dict_['basePart'].strip(),
                            'Url': f'''https://www.standardbrand.com/en/ecatalog?part={dict_['webBase'].strip()}&type=p&search=s'''} for dict_ in list_dict]).sort_values(by=['No'],
                                                                                                                                                                          ascending=[True],
                                                                                                                                                                          ignore_index=True)
df_correct.to_excel('./test_part_number.xlsx', index=False)

df_correct

Data Outputting...


,No,Part Number,Url
0,1,APK100,https://www.standardbrand.com/en/ecatalog?part...
1,2,APK101,https://www.standardbrand.com/en/ecatalog?part...
2,3,APK102,https://www.standardbrand.com/en/ecatalog?part...
3,4,APK103,https://www.standardbrand.com/en/ecatalog?part...
4,5,APS101,https://www.standardbrand.com/en/ecatalog?part...
...,...,...,...
445,446,APS717,https://www.standardbrand.com/en/ecatalog?part...
446,447,G92003,https://www.standardbrand.com/en/ecatalog?part...
447,448,G92004,https://www.standardbrand.com/en/ecatalog?part...
448,449,G92005,https://www.standardbrand.com/en/ecatalog?part...


In [7]:
print('Output Finished !')
print()

Output Finished !



In [8]:
crawler_series['Request_Url'], crawler_series['Check_Url'] = url_request, url_check

list_series_error.append(crawler_series)

list_series_error

[No                                                             1
 Url            https://www.standardbrand.com/en/ecatalog?part...
 SKU Count                                                    450
 Page                                                           5
 Request_Url    https://ecatalog.smpcorp.com/V2/STD/api/part/p...
 Check_Url      https://www.standardbrand.com/en/ecatalog?part...
 Name: 0, dtype: object]

In [9]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_url_error.xlsx', index=False)
    print('- Error Occurred -')
    print()

df_error if list_series_error else ''

- Error Occurred -



,No,Url,SKU Count,Page,Request_Url,Check_Url
0,1,https://www.standardbrand.com/en/ecatalog?part...,450,5,https://ecatalog.smpcorp.com/V2/STD/api/part/p...,https://www.standardbrand.com/en/ecatalog?part...


In [10]:
print('Done ~')

Done ~
